In [ ]:
import cv2
import os
import tensorflow as tf
import numpy as np
import time

# Create a directory to store the predicted frames
output_dir = 'C:/Users/Nabilla/Testing_Revisi/segnetwiener_siang_doubleline_revisi'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the Video
video_path = '512_new_siang_doubleline.mp4'
cap = cv2.VideoCapture(video_path)

# Load the Segmentation Model
model = tf.keras.models.load_model('segnet-50-512-revisi-wiener.hdf5')

# Define the class colors
class_colors = [(0, 0, 0), (0, 0, 128), (0, 128, 0)]  # Black, Red, Green

# Define preprocessing function
def preprocess_image(image):
    # Resize the image to match the model's input size
    image = cv2.resize(image, (512, 256))
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Run Segmentation on Frames
current_frame = 1
start_time = time.time()
while True:
    success, frame = cap.read()
    if success:
        name = os.path.join(output_dir, f'frame{current_frame}.jpg')
        pred_name = os.path.join(output_dir, f'pred{current_frame}.png')
        print(f'Creating {name}')
        
        # Preprocess frame
        preprocessed_frame = preprocess_image(frame)

        # Expand dimensions to match model input shape
        input_data = tf.expand_dims(preprocessed_frame, axis=0)

        # Make prediction
        pred = model.predict(input_data)

        # Convert prediction to single-channel mask
        pred = tf.argmax(pred, axis=-1)
        pred = tf.squeeze(pred, axis=0)

        # Apply class colors to the mask
        mask = np.zeros_like(frame)
        for class_idx, color in enumerate(class_colors):
            mask[pred == class_idx] = color

        # Save predicted mask as an image
        cv2.imwrite(pred_name, mask)

        # Save the original frame
        cv2.imwrite(name, frame)

        current_frame += 1
    else:
        break

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

cap.release()
cv2.destroyAllWindows()


Resize Video

In [ ]:
import cv2

def resize_video(input_path, output_path, new_width, new_height):
    # Open the video file
    video = cv2.VideoCapture(input_path)

    # Get the original video's width and height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a VideoWriter object to save the resized video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec (e.g., 'XVID', 'MP4V')
    fps = video.get(cv2.CAP_PROP_FPS)  # Get the original video's frames per second
    output_video = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    while True:
        # Read a frame from the video
        ret, frame = video.read()

        if not ret:
            break

        # Resize the frame to the desired width and height
        resized_frame = cv2.resize(frame, (new_width, new_height))

        # Write the resized frame to the output video file
        output_video.write(resized_frame)

    # Release the video capture and writer objects
    video.release()
    output_video.release()

    print("Resized video saved successfully!")


# Example usage
input_file = "./crop/new_siang_doubleline.mp4"
output_file = "512_new_siang_doubleline.mp4"
new_width = 512
new_height = 256

resize_video(input_file, output_file, new_width, new_height)


Merge and Overlay

In [ ]:
import cv2
import numpy as np

def overlay_videos(video1_path, video2_path, output_path):
    # Open the videos
    video1 = cv2.VideoCapture(video1_path)
    video2 = cv2.VideoCapture(video2_path)

    # Get the video properties of video1
    width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video1.get(cv2.CAP_PROP_FPS)
    num_frames = int(fps)  # Number of frames to extract (1 second)

    # Set the target size for resizing
    target_width = 512
    target_height = 256

    # Create the output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

    # Iterate over the frames
    while video1.isOpened() and video2.isOpened():
        ret1, frame1 = video1.read()
        ret2, frame2 = video2.read()

        if not ret1 or not ret2:
            break

        # Resize video1 frames to the target size
        frame1_resized = cv2.resize(frame1, (target_width, target_height))

        # Resize video2 frames to the target size
        frame2_resized = cv2.resize(frame2, (target_width, target_height))

        # Remove black pixels from video1 frames
        frame1_resized[np.all(frame1_resized == [0, 0, 0], axis=2)] = [255, 255, 255]

        # Overlay video1 frames on video2
        blended_frame = cv2.addWeighted(frame1_resized, 0.5, frame2_resized, 0.5, 0)

        # Write the modified frame to the output video
        output.write(blended_frame)

        # Display the frame
        cv2.imshow('Overlay', blended_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the resources
    video1.release()
    video2.release()
    output.release()
    cv2.destroyAllWindows()

# Specify the paths of the input videos and the output video
video1_path = "C:/Users/Nabilla/TUGAS AKHIR/Testing/UNET/FRAMES_ONLY.mp4"
video2_path = 'C:/Users/Nabilla/TUGAS AKHIR/Testing/UNET/PNG_ONLY.mp4'
output_path = 'C:/Users/Nabilla/TUGAS AKHIR/Testing/UNET/UNET.mp4'

# Overlay the videos and remove black color from the first second of video1
overlay_videos(video1_path, video2_path, output_path)


Frames to Video

In [ ]:
import cv2
import os

# Path to the directory containing the frames
frames_directory = r"C:\Users\Nabilla\TUGAS AKHIR\Testing\UNET\unet_siang_doubleline_FRAMES_ONLY"

# Output video path and filename
output_video = "C:/Users/Nabilla/TUGAS AKHIR/Testing/UNET/FRAMES_ONLY.mp4"

# Get the list of frames in the directory
frame_files = sorted(os.listdir(frames_directory))

# Set the video dimensions and FPS
frame = cv2.imread(os.path.join(frames_directory, frame_files[0]))
height, width, _ = frame.shape
fps = 24  # Adjust the frames per second as needed

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Specify the video codec
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Iterate over each frame file and write it to the video
for frame_file in frame_files:
    frame_path = os.path.join(frames_directory, frame_file)
    frame = cv2.imread(frame_path)
    video_writer.write(frame)

# Release the video writer and close the video file
video_writer.release()

print("Video creation complete!")
